In [1]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.python.client import device_lib
import pandas as pd
from tqdm.auto import tqdm
from PIL import Image
import time
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from math import isnan
from numpy import average
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils import resample
import warnings
size = 224
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


2024-02-26 12:01:23.231528: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-26 12:01:23.861714: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def diceloss(pred, target,threshold, num_classes=4):
    smooth = 1.
    dice_per_class = np.zeros(num_classes)
    
    for class_id in range(num_classes):
        pred_class = pred[class_id, ...]
        target_class = target[class_id, ...]

        pred_int_class=np.where(pred_class>threshold,1,0)
    
        TP_pixel=np.sum(pred_int_class * target_class)
        FP_pixel= np.sum(target_class *(1-(pred_int_class * target_class)))
        FN_pixel= np.sum(pred_int_class *(1-(pred_int_class * target_class)))

        dice_per_class[class_id] = 1 - (2. * TP_pixel) / (2. * TP_pixel + FP_pixel+FN_pixel)

    return np.mean(dice_per_class)

def keras2TFlite(model_path,name):
    #load a pre-trained model
    with tf.device('/gpu:0'):
        model = tf.keras.models.load_model(model_path)
        #convert the model
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        #save the converted model
        with open('../../model/classification/tflite/'+name+'.tflite', 'wb') as f:
            f.write(tflite_model)
        print('Model converted successfully')

def createDirectory(directory):
    """_summary_
        create Directory
    Args:
        directory (string): file_path
    """    
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("Error: Failed to create the directory.")

def TFLiteInference(model_path, name, x_test, y_test):
    # Load TFLite model and allocate tensors.
    interpreter = tf.lite.Interpreter(model_path=model_path)
    interpreter.allocate_tensors()
    with tf.device('/gpu:0'):
        input_index = interpreter.get_input_details()[0]['index']
        output_index  = interpreter.get_output_details()[0]['index']
        
        sum_correct = 0
        sum_time=0
        predict=np.zeros((len(x_test),4,size,size),dtype=np.float32)
        for idx, data in tqdm(enumerate(zip(x_test,y_test))):
            img = data[0]
            label = data[1]
            img= img.astype(np.float32)
            img = tf.expand_dims(img, axis=0)
            
            interpreter.set_tensor(input_index, img)
            start_time = time.time()
            interpreter.invoke()
            
            output = interpreter.get_tensor(output_index)
            stop_time = time.time()
            sum_time += stop_time-start_time
            
            sum_correct+=1-diceloss(output[0], label,0.5, num_classes=4)
            predict[idx]=output
            mean_acc=sum_correct/float(idx+1)
            mean_time=sum_time/float(idx+1)
        print('Model: ',name)
        print('Mean dice: ',mean_acc)
        print('Mean Inference Time: ',mean_time)
        return predict
# keras2TFlite('../../model/classification/MobileNetV2_checkpoints.h5','MobileNetV2')
# keras2TFlite('../../model/classification/MobileNetV1_acc_checkpoints.h5','MobileNetV1')
# keras2TFlite('../../model/classification/DenseNet121_checkpoints.h5','DenseNet121')
# keras2TFlite('../../model/classification/EfficientNetV2B0_checkpoints.h5','EfficientNetV2B0')
# keras2TFlite('../../model/classification/NASNetMobile_checkpoints.h5','NASNetMobile')
# keras2TFlite('../../model/classification/ResNet50_checkpoints.h5','ResNet50')        

In [3]:
test_df = pd.read_csv(
    '../../data/segmentationDDH/test_aug_segmentation_dataset.csv')

test_img_list = test_df['file name'].to_list()
test_label_list = test_df['standard mask'].to_list()
test_case_list = test_df['case'].to_list()
test_img_path = '../../data/segmentationDDH/aug_dataset/test/'
   
x_test=np.zeros((len(test_img_list),3,size,size),dtype=np.float32)
y_test=np.zeros((len(test_img_list),4,size,size),dtype=np.float32)

for i in tqdm(range(len(test_img_list))):
    x_test[i] = np.swapaxes(np.swapaxes(np.array(Image.open(
        test_img_path+str(test_case_list[i])+'/image/'+test_img_list[i]).resize((size, size))),1,2),0,1)
    y_test[i,0]=np.array(Image.open(
        test_img_path+str(test_case_list[i])+'/mask/'+str(test_label_list[i]).zfill(5)+'/1.png').resize((size, size)))
    y_test[i,1]=np.array(Image.open(
        test_img_path+str(test_case_list[i])+'/mask/'+str(test_label_list[i]).zfill(5)+'/2.png').resize((size, size)))
    y_test[i,2]=np.array(Image.open(
        test_img_path+str(test_case_list[i])+'/mask/'+str(test_label_list[i]).zfill(5)+'/3.png').resize((size, size)))
    y_test[i,3]=np.array(Image.open(
        test_img_path+str(test_case_list[i])+'/mask/'+str(test_label_list[i]).zfill(5)+'/4.png').resize((size, size)))

x_test = x_test/255.0
y_test = y_test/255.0

  0%|          | 0/584 [00:00<?, ?it/s]

In [4]:
DeepLabV3Plus_pre=TFLiteInference('../../model/segmentation/tflite/DeepLabV3Plus.tflite','DeepLabV3Plus',x_test,y_test)
NestedUNet_pre=TFLiteInference('../../model/segmentation/tflite/NestedUNet.tflite','NestedUNet',x_test,y_test)
PAN_pre=TFLiteInference('../../model/segmentation/tflite/PAN.tflite','PAN',x_test,y_test)
UNet_pre=TFLiteInference('../../model/segmentation/tflite/UNet.tflite','UNet',x_test,y_test)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
2024-02-26 12:01:39.109008: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38158 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:01:00.0, compute capability: 8.0


0it [00:00, ?it/s]

Model:  DeepLabV3Plus
Mean dice:  0.6241230808165933
Mean Inference Time:  0.07776492262539798


0it [00:00, ?it/s]

Model:  NestedUNet
Mean dice:  0.5977729413596834
Mean Inference Time:  0.18499356916505996


0it [00:00, ?it/s]

Model:  PAN
Mean dice:  0.5657209072467786
Mean Inference Time:  0.0376259251816632


0it [00:00, ?it/s]

Model:  UNet
Mean dice:  0.6094783270514451
Mean Inference Time:  0.13916936312636283


In [5]:
import math

def calculate_standard_error(p, n):
    return math.sqrt((p * (1 - p)) / n)
dsc_optim1=0
dsc_optim2=0
dsc_optim3=0
dsc_optim4=0
threshold1=0
threshold2=0
threshold3=0
threshold4=0
dsc1_1=[]
dsc1_2=[]
dsc2_1=[]
dsc2_2=[]
dsc3_1=[]
dsc3_2=[]
dsc4_1=[]
dsc4_2=[]
threshold_list=[]
def total_performance(pred, target, threshold, num_classes=2):
    dice_per_class = np.zeros(num_classes)
    sensitivity_per_class = np.zeros(num_classes)
    specificity_per_class = np.zeros(num_classes)
    precision_per_class = np.zeros(num_classes)
    f1_per_class = np.zeros(num_classes)

    for class_id in range(num_classes):
        pred_class = pred[:, class_id, ...]
        target_class = target[:, class_id, ...]

        pred_int_class = np.where(pred_class >= threshold, 1, 0)

        TP_pixel = np.sum(pred_int_class * target_class)
        FN_pixel = np.sum(pred_int_class * (1 - target_class))
        FP_pixel = np.sum((1 - pred_int_class) * target_class)
        TN_pixel = np.sum((1 - pred_int_class) * (1 - target_class))

        # Dice Loss
        dice_per_class[class_id] = (2. * TP_pixel) / (2. * TP_pixel + FN_pixel + FP_pixel)

        # Sensitivity
        sensitivity_per_class[class_id] = TP_pixel / (TP_pixel + FN_pixel)

        # Specificity
        specificity_per_class[class_id] = TN_pixel / (TN_pixel + FP_pixel)

        # Precision
        precision_per_class[class_id] = TP_pixel / (TP_pixel + FP_pixel)

        # F1 Score
        f1_per_class[class_id] = 2 * (precision_per_class[class_id] * sensitivity_per_class[class_id]) / (
                precision_per_class[class_id] + sensitivity_per_class[class_id])

    return dice_per_class, sensitivity_per_class, specificity_per_class, precision_per_class, f1_per_class


for i in tqdm(np.arange(-0.02,1.4,0.01)):
    dice1,sensitivity1,specificity1, precision1, f1_1=total_performance(NestedUNet_pre[:], y_test[:],i)
    dice1=dice1
    sensitivity1=sensitivity1
    dsc1_1.append(dice1[0].item())
    dsc1_2.append(dice1[1].item())
    dice2,sensitivity2,specificity2, precision2, f2_1=total_performance(UNet_pre[:], y_test[:],i)
    dice2=dice2
    sensitivity2=sensitivity2
    dsc2_1.append(dice2[0].item())
    dsc2_2.append(dice2[1].item())
    dice3,sensitivity3,specificity3, precision3, f3_1=total_performance(DeepLabV3Plus_pre[:], y_test[:],i)
    dice3=dice3
    sensitivity1=sensitivity1
    dsc3_1.append(dice3[0].item())
    dsc3_2.append(dice3[1].item())
    dice4,sensitivity4,specificity4, precision4, f4_1=total_performance(PAN_pre [:], y_test[:],i)
    dice4=dice4
    sensitivity4=sensitivity4
    dsc4_1.append(dice4[0].item())
    dsc4_2.append(dice4[1].item())
    threshold_list.append(i)
    if ((dice1[0].item()+dice1[1].item())/2)>=dsc_optim1:
        dsc_optim1=((dice1[0].item()+dice1[1].item())/2)
        threshold1=i
        
    if ((dice2[0].item()+dice2[1].item())/2)>=dsc_optim2:
        dsc_optim2=((dice2[0].item()+dice2[1].item())/2)
        threshold2=i
        
    if ((dice3[0].item()+dice3[1].item())/2)>=dsc_optim3:
        dsc_optim3=((dice3[0].item()+dice3[1].item())/2)
        threshold3=i
        
    if ((dice4[0].item()+dice4[1].item())/2)>=dsc_optim4:
        dsc_optim4=((dice4[0].item()+dice4[1].item())/2)
        threshold4=i
# 표본 크기가 필요하다면 데이터에서 얻을 수 있습니다.
n_NestedUNet = len(NestedUNet_pre)
n_UNet = len(UNet_pre)
n_DeepLabV3Plus = len(DeepLabV3Plus_pre)
n_PAN = len(PAN_pre )

# 성능 지표에 대한 확률 추정
p_NestedUNet = dsc_optim1  # 또는 NestedUNet_sensitivity
p_UNet = dsc_optim2  # 또는 UNet_sensitivity
p_DeepLabV3Plus = dsc_optim3  # 또는 NestedUNet_sensitivity
p_PAN = dsc_optim4  # 또는 UNet_sensitivity

# 95% 신뢰구간 계산
NestedUNet_dice_ci = (p_NestedUNet - 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet),
                  p_NestedUNet + 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet))

UNet_dice_ci = (p_UNet - 1.96 * calculate_standard_error(p_UNet, n_UNet),
                   p_UNet + 1.96 * calculate_standard_error(p_UNet, n_UNet))

DeepLabV3Plus_dice_ci = (p_DeepLabV3Plus - 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus),
                  p_DeepLabV3Plus + 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus))

PAN_dice_ci = (p_PAN - 1.96 * calculate_standard_error(p_PAN, n_PAN),
                   p_PAN + 1.96 * calculate_standard_error(p_PAN, n_PAN))

dice1,sensitivity1,specificity1, precision1, f1_1=total_performance(NestedUNet_pre, y_test,threshold1)
dice2,sensitivity2,specificity2, precision2, f1_2=total_performance(UNet_pre, y_test,threshold2)
dice3,sensitivity3,specificity3, precision3, f1_3=total_performance(DeepLabV3Plus_pre, y_test,threshold3)
dice4,sensitivity4,specificity4, precision4, f1_4=total_performance(PAN_pre , y_test,threshold4)
p_NestedUNet = ((sensitivity1[0].item()+sensitivity1[1].item())/2)
p_UNet = ((sensitivity2[0].item()+sensitivity2[1].item())/2)
p_DeepLabV3Plus = ((sensitivity3[0].item()+sensitivity3[1].item())/2)
p_PAN = ((sensitivity4[0].item()+sensitivity4[1].item())/2)

NestedUNet_sensitivity_ci = (p_NestedUNet - 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet),
                  p_NestedUNet + 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet))

UNet_sensitivity_ci = (p_UNet - 1.96 * calculate_standard_error(p_UNet, n_UNet),
                   p_UNet + 1.96 * calculate_standard_error(p_UNet, n_UNet))
DeepLabV3Plus_sensitivity_ci = (p_DeepLabV3Plus - 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus),
                  p_DeepLabV3Plus + 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus))

PAN_sensitivity_ci = (p_PAN - 1.96 * calculate_standard_error(p_PAN, n_PAN),
                   p_PAN + 1.96 * calculate_standard_error(p_PAN, n_PAN))

p_NestedUNet = ((specificity1[0].item()+specificity1[1].item())/2)
p_UNet = ((specificity2[0].item()+specificity2[1].item())/2)
p_DeepLabV3Plus = ((specificity3[0].item()+specificity3[1].item())/2)
p_PAN = ((specificity4[0].item()+specificity4[1].item())/2)

NestedUNet_specificity_ci = (p_NestedUNet - 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet),
                  p_NestedUNet + 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet))

UNet_specificity_ci = (p_UNet - 1.96 * calculate_standard_error(p_UNet, n_UNet),
                   p_UNet + 1.96 * calculate_standard_error(p_UNet, n_UNet))
DeepLabV3Plus_specificity_ci = (p_DeepLabV3Plus - 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus),
                  p_DeepLabV3Plus + 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus))

PAN_specificity_ci = (p_PAN - 1.96 * calculate_standard_error(p_PAN, n_PAN),
                   p_PAN + 1.96 * calculate_standard_error(p_PAN, n_PAN))

p_NestedUNet = ((precision1[0].item()+precision1[1].item())/2)
p_UNet = ((precision2[0].item()+precision2[1].item())/2)
p_DeepLabV3Plus = ((precision3[0].item()+precision3[1].item())/2)
p_PAN = ((precision4[0].item()+precision4[1].item())/2)

NestedUNet_precision_ci = (p_NestedUNet - 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet),
                  p_NestedUNet + 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet))

UNet_precision_ci = (p_UNet - 1.96 * calculate_standard_error(p_UNet, n_UNet),
                   p_UNet + 1.96 * calculate_standard_error(p_UNet, n_UNet))
DeepLabV3Plus_precision_ci = (p_DeepLabV3Plus - 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus),
                  p_DeepLabV3Plus + 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus))

PAN_precision_ci = (p_PAN - 1.96 * calculate_standard_error(p_PAN, n_PAN),
                   p_PAN + 1.96 * calculate_standard_error(p_PAN, n_PAN))


p_NestedUNet = ((f1_1[0].item()+f1_1[1].item())/2)
p_UNet = ((f1_2[0].item()+f1_2[1].item())/2)
p_DeepLabV3Plus = ((f1_3[0].item()+f1_3[1].item())/2)
p_PAN = ((f1_4[0].item()+f1_4[1].item())/2)

NestedUNet_f1_ci = (p_NestedUNet - 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet),
                  p_NestedUNet + 1.96 * calculate_standard_error(p_NestedUNet, n_NestedUNet))

UNet_f1__ci = (p_UNet - 1.96 * calculate_standard_error(p_UNet, n_UNet),
            p_UNet + 1.96 * calculate_standard_error(p_UNet, n_UNet))
DeepLabV3Plus_f1_ci = (p_DeepLabV3Plus - 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus),
                  p_DeepLabV3Plus + 1.96 * calculate_standard_error(p_DeepLabV3Plus, n_DeepLabV3Plus))

PAN_f1_ci = (p_PAN - 1.96 * calculate_standard_error(p_PAN, n_PAN),
                   p_PAN + 1.96 * calculate_standard_error(p_PAN, n_PAN))

print(f"NestedUNet: Dice-Coeffidence = {dsc_optim1:.3f}({NestedUNet_dice_ci[0]:.3f}-{NestedUNet_dice_ci[1]:.3f}) RE = {(NestedUNet_sensitivity_ci[0]+NestedUNet_sensitivity_ci[1])/2:.3f}({NestedUNet_sensitivity_ci[0]:.3f}-{NestedUNet_sensitivity_ci[1]:.3f}) PR = {(NestedUNet_precision_ci[0]+NestedUNet_precision_ci[1])/2:.3f}({NestedUNet_precision_ci[0]:.3f}-{NestedUNet_precision_ci[1]:.3f}) SP  = {(NestedUNet_specificity_ci[0]+NestedUNet_specificity_ci[1])/2:.3f}({NestedUNet_specificity_ci[0]:.3f}-{NestedUNet_specificity_ci[1]:.3f})")
print(f"UNet: Dice-Coeffidence = {dsc_optim2:.3f}({UNet_dice_ci[0]:.3f}-{UNet_dice_ci[1]:.3f}) RE = {(UNet_sensitivity_ci[0]+UNet_sensitivity_ci[1])/2:.3f}({UNet_sensitivity_ci[0]:.3f}-{UNet_sensitivity_ci[1]:.3f}) PR = {(UNet_precision_ci[0]+UNet_precision_ci[1])/2:.3f}({UNet_precision_ci[0]:.3f}-{UNet_precision_ci[1]:.3f}) SP  = {(UNet_specificity_ci[0]+UNet_specificity_ci[1])/2:.3f}({UNet_specificity_ci[0]:.3f}-{UNet_specificity_ci[1]:.3f})")
print(f"DeepLabV3Plus: Dice-Coeffidence = {dsc_optim3:.3f}({DeepLabV3Plus_dice_ci[0]:.3f}-{DeepLabV3Plus_dice_ci[1]:.3f}) RE = {(DeepLabV3Plus_sensitivity_ci[0]+DeepLabV3Plus_sensitivity_ci[1])/2:.3f}({DeepLabV3Plus_sensitivity_ci[0]:.3f}-{DeepLabV3Plus_sensitivity_ci[1]:.3f}) PR = {(DeepLabV3Plus_precision_ci[0]+DeepLabV3Plus_precision_ci[1])/2:.3f}({DeepLabV3Plus_precision_ci[0]:.3f}-{DeepLabV3Plus_precision_ci[1]:.3f}) SP  = {(DeepLabV3Plus_specificity_ci[0]+DeepLabV3Plus_specificity_ci[1])/2:.3f}({DeepLabV3Plus_specificity_ci[0]:.3f}-{DeepLabV3Plus_specificity_ci[1]:.3f})")
print(f"PAN: Dice-Coeffidence = {dsc_optim4:.3f}({PAN_dice_ci[0]:.3f}-{PAN_dice_ci[1]:.3f}) RE = {(PAN_sensitivity_ci[0]+PAN_sensitivity_ci[1])/2:.3f}({PAN_sensitivity_ci[0]:.3f}-{PAN_sensitivity_ci[1]:.3f}) PR = {(PAN_precision_ci[0]+PAN_precision_ci[1])/2:.3f}({PAN_precision_ci[0]:.3f}-{PAN_precision_ci[1]:.3f}) SP  = {(PAN_specificity_ci[0]+PAN_specificity_ci[1])/2:.3f}({PAN_specificity_ci[0]:.3f}-{PAN_specificity_ci[1]:.3f})")

  0%|          | 0/142 [00:00<?, ?it/s]

/tmp/ipykernel_3823004/820554432.py:44: RuntimeWarning: invalid value encountered in scalar divide
  sensitivity_per_class[class_id] = TP_pixel / (TP_pixel + FN_pixel)


NestedUNet: Dice-Coeffidence = 0.664(0.625-0.702) RE = 0.625(0.585-0.664) PR = 0.713(0.677-0.750) SP  = 0.998(0.995-1.002)
UNet: Dice-Coeffidence = 0.664(0.626-0.702) RE = 0.619(0.580-0.658) PR = 0.717(0.681-0.754) SP  = 0.998(0.995-1.002)
DeepLabV3Plus: Dice-Coeffidence = 0.691(0.653-0.728) RE = 0.646(0.607-0.685) PR = 0.743(0.707-0.778) SP  = 0.998(0.995-1.002)
PAN: Dice-Coeffidence = 0.646(0.608-0.685) RE = 0.612(0.572-0.651) PR = 0.688(0.651-0.726) SP  = 0.998(0.995-1.002)
